In [1]:
import mne
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
brown_csv = 'Brown.csv'
pink_csv = 'Pink.csv'
silent_csv = 'Silent.csv'
white_csv = 'White.csv'

In [ ]:
# read brown_csv using pandas. The header is on the second row. Just skip the first row
brown_df = pd.read_csv(brown_csv, header=1)

In [ ]:
print(brown_df.columns)
# print length of dataframe row
print(len(brown_df))

#### List of EEG Electrode
EEG.AF3, EEG.F7, EEG.F3, EEG.FC5, EEG.T7, EEG.P7, EEG.O1, EEG.O2, EEG.P8, EEG.T8, EEG.FC6, EEG.F4, EEG.F8, EEG.AF4

In [ ]:
# Create a new dataframe but only copy this columns: EEG.AF3, EEG.F7, EEG.F3, EEG.FC5, EEG.T7, EEG.P7, EEG.O1, EEG.O2, EEG.P8, EEG.T8, EEG.FC6, EEG.F4, EEG.F8, EEG.AF4
brown_df_signals = brown_df[['EEG.AF3', 'EEG.F7', 'EEG.F3', 'EEG.FC5', 'EEG.T7', 'EEG.P7', 'EEG.O1', 'EEG.O2', 'EEG.P8', 'EEG.T8', 'EEG.FC6', 'EEG.F4', 'EEG.F8', 'EEG.AF4']]

In [ ]:
# change the header name from 1 to 14
brown_df_signals.columns = [1,2,3,4,5,6,7,8,9,10,11,12,13,14]
brown_df_signals.head()

In [ ]:
# plot all 14 signals using for loop and subplot of matplotlib
plt.figure(figsize=(20,20))
for i in range(14):
    plt.subplot(14,1,i+1)
    plt.plot(brown_df_signals[i+1])
    plt.title(i+1)

# horizontal space
plt.subplots_adjust(hspace=1)

In [ ]:
# remove first 10 seconds of data and last 10 seconds of data
fs = 256
brown_df_signals = brown_df_signals[fs*10:-fs*10]

# print the length of dataframe row
print(f"Length of dataframe row: {len(brown_df_signals)}")

In [ ]:
# transpose the dataframe
brown_df_signals = brown_df_signals.T

In [ ]:
print(f"Shape of dataframe: {brown_df_signals.shape}")

In [ ]:
info = mne.create_info(brown_df_signals.shape[0], fs, ch_types='eeg')
raw = mne.io.RawArray(brown_df_signals, info)

# Kode Beneran

In [2]:
def read_csv_and_create_raw(csv_file, sfreq=256):
    # Read CSV file
    data = pd.read_csv(csv_file, header=1)

    data = data[['EEG.AF3', 'EEG.F7', 'EEG.F3', 'EEG.FC5', 'EEG.T7', 'EEG.P7', 'EEG.O1', 'EEG.O2', 'EEG.P8', 'EEG.T8', 'EEG.FC6', 'EEG.F4', 'EEG.F8', 'EEG.AF4']]

    data.columns = [1,2,3,4,5,6,7,8,9,10,11,12,13,14]

    fs = 256
    data = data[fs*10:-fs*10]

    data = data.T

    # Create an Info object with the simulation parameters
    info = mne.create_info(data.shape[0], sfreq, ch_types='eeg')

    # Create a Raw object with the data
    raw = mne.io.RawArray(data, info)

    return raw

In [3]:
csv_files = {'brown': 'Brown.csv', 'White': 'White.csv', 'Pink': 'Pink.csv'}
raws = {name: read_csv_and_create_raw(file) for name, file in csv_files.items()}

Creating RawArray with float64 data, n_channels=14, n_times=25600
    Range : 0 ... 25599 =      0.000 ...    99.996 secs
Ready.
Creating RawArray with float64 data, n_channels=14, n_times=25600
    Range : 0 ... 25599 =      0.000 ...    99.996 secs
Ready.
Creating RawArray with float64 data, n_channels=14, n_times=25600
    Range : 0 ... 25599 =      0.000 ...    99.996 secs
Ready.


In [4]:
# Calculate the maximum possible tmax_stimuli value based on the length of the data
max_time = raws[next(iter(raws))].times[-1]

# Set the baseline and stimuli periods
tmin_baseline, tmax_baseline = 0, 50
tmin_stimuli, tmax_stimuli = 50, max_time

# Define the frequency bands
freq_bands = {'delta': (0.5, 4),
              'theta': (4, 8),
              'alpha': (8, 13),
              'beta': (13, 30),
              'gamma': (30, 50)}

# Calculate the relative power of the alpha band for each condition
relative_alpha_powers = {}

for name, raw in raws.items():
    # Extract the baseline and stimuli periods
    raw_baseline = raw.copy().crop(tmin_baseline, tmax_baseline)
    raw_stimuli = raw.copy().crop(tmin_stimuli, tmax_stimuli)

    # Calculate the power spectral density for the baseline and stimuli periods without filtering
    psd_baseline_total, freqs = mne.time_frequency.psd_array_multitaper(raw_baseline.get_data(), sfreq=raw_baseline.info['sfreq'])
    psd_stimuli_total, _ = mne.time_frequency.psd_array_multitaper(raw_stimuli.get_data(), sfreq=raw_stimuli.info['sfreq'])

    # Filter the data for the alpha band
    raw_baseline.filter(freq_bands['alpha'][0], freq_bands['alpha'][1], fir_design='firwin')
    raw_stimuli.filter(freq_bands['alpha'][0], freq_bands['alpha'][1], fir_design='firwin')

    # Calculate the power spectral density for the baseline and stimuli periods in the alpha band
    psd_baseline_alpha, freqs = mne.time_frequency.psd_array_multitaper(raw_baseline.get_data(), sfreq=raw_baseline.info['sfreq'], fmin=freq_bands['alpha'][0], fmax=freq_bands['alpha'][1])
    psd_stimuli_alpha, _ = mne.time_frequency.psd_array_multitaper(raw_stimuli.get_data(), sfreq=raw_stimuli.info['sfreq'], fmin=freq_bands['alpha'][0], fmax=freq_bands['alpha'][1])

    # Calculate the total power of the baseline and stimuli periods
    total_power_baseline = np.sum(psd_baseline_total)
    total_power_stimuli = np.sum(psd_stimuli_total)

    # Calculate the relative power of the alpha band for the baseline and stimuli periods
    relative_alpha_power_baseline = np.sum(psd_baseline_alpha) / total_power_baseline
    relative_alpha_power_stimuli = np.sum(psd_stimuli_alpha) / total_power_stimuli

    relative_alpha_powers[name] = (relative_alpha_power_baseline, relative_alpha_power_stimuli)


# Print the results
print("Relative Alpha Power:")
for name, powers in relative_alpha_powers.items():
    print(f"{name.capitalize()} noise:")
    print(f"  Baseline: {powers[0]:.4f}")
    print(f"  Stimuli: {powers[1]:.4f}")

    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 8 - 13 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 8.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 7.00 Hz)
- Upper passband edge: 13.00 Hz
- Upper transition bandwidth: 3.25 Hz (-6 dB cutoff frequency: 14.62 Hz)
- Filter length: 423 samples (1.652 sec)

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 8 - 13 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
-

In [7]:
# Calculate the maximum possible tmax_stimuli value based on the length of the data
max_time = raws[next(iter(raws))].times[-1]

# Set the baseline and stimuli periods
tmin_baseline, tmax_baseline = 0, 50
tmin_stimuli, tmax_stimuli = 50, max_time

# Define the frequency bands
freq_bands = {'delta': (0.5, 4),
              'theta': (4, 8),
              'alpha': (8, 13),
              'beta': (13, 30),
              'gamma': (30, 50)}

# Calculate the relative power of the alpha band for each condition
relative_alpha_powers = {}

for name, raw in raws.items():
    # Extract the baseline and stimuli periods
    raw_baseline = raw.copy().crop(tmin_baseline, tmax_baseline)
    raw_stimuli = raw.copy().crop(tmin_stimuli, tmax_stimuli)

    # Calculate the power spectral density for the baseline and stimuli periods without filtering
    psd_baseline_total, freqs = mne.time_frequency.psd_array_multitaper(raw_baseline.get_data(), sfreq=raw_baseline.info['sfreq'])
    psd_stimuli_total, _ = mne.time_frequency.psd_array_multitaper(raw_stimuli.get_data(), sfreq=raw_stimuli.info['sfreq'])

    # Filter the data for the alpha band
    raw_baseline.filter(freq_bands['beta'][0], freq_bands['beta'][1], fir_design='firwin')
    raw_stimuli.filter(freq_bands['beta'][0], freq_bands['beta'][1], fir_design='firwin')

    # Calculate the power spectral density for the baseline and stimuli periods in the alpha band
    psd_baseline_alpha, freqs = mne.time_frequency.psd_array_multitaper(raw_baseline.get_data(), sfreq=raw_baseline.info['sfreq'], fmin=freq_bands['alpha'][0], fmax=freq_bands['alpha'][1])
    psd_stimuli_alpha, _ = mne.time_frequency.psd_array_multitaper(raw_stimuli.get_data(), sfreq=raw_stimuli.info['sfreq'], fmin=freq_bands['alpha'][0], fmax=freq_bands['alpha'][1])

    # Calculate the total power of the baseline and stimuli periods
    total_power_baseline = np.sum(psd_baseline_total)
    total_power_stimuli = np.sum(psd_stimuli_total)

    # Calculate the relative power of the alpha band for the baseline and stimuli periods
    relative_alpha_power_baseline = np.sum(psd_baseline_alpha) / total_power_baseline
    relative_alpha_power_stimuli = np.sum(psd_stimuli_alpha) / total_power_stimuli

    relative_alpha_powers[name] = (relative_alpha_power_baseline, relative_alpha_power_stimuli)


# Print the results
print("Relative Alpha Power:")
for name, powers in relative_alpha_powers.items():
    print(f"{name.capitalize()} noise:")
    print(f"  Baseline: {powers[0]:.4f}")
    print(f"  Stimuli: {powers[1]:.4f}")

    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 13 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 13.00
- Lower transition bandwidth: 3.25 Hz (-6 dB cutoff frequency: 11.38 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 261 samples (1.020 sec)

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 13 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuati